In [ ]:
import glob
import re
import numpy as np
from numpy import genfromtxt
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 5]

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [ ]:
outfiles = glob.glob('../out/*.timers.*')
outfiles = natural_sort(outfiles)
numout = len(outfiles)

In [ ]:
# Read timer structures
my_data = genfromtxt(outfiles[0], delimiter=",")
timer_shape = np.shape(my_data)
num_proc = timer_shape[0]
num_timers = timer_shape[1]-1
print("Found "+str(num_proc)+" processors sharing "+str(num_timers)+" timers from "+str(numout)+" files.")

In [ ]:
# Load timers in a Numpy array (t_iter, timer_entry, cpu)
timer_data = np.zeros((numout,num_timers,num_proc))
for i,file in enumerate(outfiles):
    my_data = genfromtxt(file, delimiter=",")
    timer_data[i] = np.transpose(my_data[:,1:])

timer_names = ['T_ITER','T_DISTR','T_FORCES','T_EVOL',
               'T_LEAFSYNC','T_TREEUPDATE','T_OUTPUT']

In [ ]:
# Total times. Sum timers per iteration and cpu
total_per_task = np.sum(timer_data,axis=(0,2))
total_cumul = np.sum(total_per_task[0])
total_cumul_diagnostic = np.sum( total_per_task[1:] )
total_per_cpu = np.sum(timer_data,axis=(0,1))
total_per_iter = np.sum(timer_data,axis=(1,2))

print("""Total time (CPU time, T_ITER): """+str(total_cumul)+"""s (all processes measure sync barrier at output)
Total time (CPU time, sum per task): """+str(total_cumul_diagnostic)+"""s (master measures sync barrier at output)
Guessed output time (with two lines above):"""+str(total_cumul_diagnostic-total_cumul)+"""s
Measured output time (T_OUPUT): """+str(total_per_task[-1])+"""s""")

In [ ]:
fig_per_cpu = plt.figure();
plt.bar(range(0,4), total_per_cpu)
plt.title("Time per CPU")
plt.show()

fig_per_task = plt.figure();
plt.bar(range(1,num_timers), total_per_task[1:])
plt.xticks(range(1,num_timers), timer_names[1:], rotation=90)
plt.title("Time per task")
plt.show()

fig_per_iter = plt.figure();
ax = plt.subplot(111)
ax.semilogy( range(2,numout+1), np.sum(timer_data[1:,1:],axis=2) )
plt.xlabel("Iteration")
plt.ylabel("Time [s]")
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(timer_names[1:], loc="center left",bbox_to_anchor=(1, 0.5))
plt.show()